## BigQuery 테스트

In [1]:
from google.cloud import bigquery
from google.cloud import bigquery_storage
from google.oauth2 import service_account
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get environment variables
project_id = os.getenv("PROJECT_ID")
credentials_path = os.getenv("CREDENTIALS_PATH")

# Local에 저장된 json 파일 불러오기
credentials = service_account.Credentials.from_service_account_file(credentials_path)

In [2]:
client = bigquery.Client(credentials= credentials, project=project_id)
bqstorage_client = bigquery_storage.BigQueryReadClient(credentials=credentials)

df = client.query(f'''
    SELECT *
      FROM `{project_id}.kaggle.train`''').to_dataframe(bqstorage_client=bqstorage_client)

print(df.head())

     Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   165          40       RM         40.0     5400   Pave  Pave      Reg   
1   590          40       RM         50.0     9100   Pave  None      Reg   
2   874          40       RL         60.0    12144   Pave  None      Reg   
3  1271          40       RL          NaN    23595   Pave  None      Reg   
4  1333          20       RL         67.0     8877   Pave  None      Reg   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0   None  None        None       0     10   
1         Lvl    AllPub  ...        0   None  None        Shed     600      8   
2         Lvl    AllPub  ...        0   None  None        Othr       0      9   
3         Low    AllPub  ...        0   None  None        None       0      4   
4         Lvl    AllPub  ...        0   None  None        None       0      5   

  YrSold  SaleType  SaleCondition  SalePrice  
0   2007 

### 데이터 가공

In [3]:
non_missing_columns = df.columns[df.notna().all()].tolist()
df2 = df[non_missing_columns]
df2.shape

(1460, 62)

### 가공된 데이터 업로드

In [4]:
import pandas_gbq
table_id = 'kaggle.transformed_train'
pandas_gbq.to_gbq(df2, table_id, project_id=project_id)